# VisCPU: data pre-processing

TODO:
- write about;
- why this notebook;

## Dependencies and imports

Install and import required packages.

In [ ]:
!pip install pandas
!pip install psutil
!conda install -c plotly plotly-orca -y

In [61]:
import json
import pandas as pd
from viscpu import utils

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load datasets

Load the two datasets that will be compared. We use the first dataset as base for comparison. For example: if the first experiment has 10 cache misses and the second has 15 cache misses, this means that the number of cache misses increased.

In [13]:
dataset_1 = "../applications/simple-ff-test/data/perf-test-1.csv"
dataset_2 = "../applications/simple-ff-test/data/perf-test-2.csv"

column_names=["time", "cpu", "counter_value", "ignore_1", "event",
              "ignore_2", "ignore_3", "ignore_4", "ignore_5", "ignore_6"]
usecols=["time", "cpu", "counter_value", "event"]

df_1 = pd.read_csv(dataset_1, skiprows=1, header=None, names=column_names, usecols=usecols)
df_2 = pd.read_csv(dataset_2, skiprows=1, header=None, names=column_names, usecols=usecols)

Define captured events:

In [6]:
events = ["cpu-cycles", "cache-misses", "instructions", "cycle_activity.cycles_l3_miss"]

## Pre-processing

In [125]:
output = {}

Create the setup of the CPUs. Here you can choose how many CPUs will be shown on each row.

In [126]:
cpu_labels, cpu_setup = utils.get_cpu_setup(df_1["cpu"].unique(), cpus_per_row=4)

output["cpu_labels"] = cpu_labels
output["cpu_setup"] = cpu_setup

Load data from each captured event and write in `output`:

In [127]:
for event in events:
    output["dataset-1"] = {}
    times, captures = utils.get_event_data(df_1, cpu_setup, event)
    output["dataset-1"][event] = {"captures": captures}
    
    output["dataset-2"] = {}
    times, captures = utils.get_event_data(df_2, cpu_setup, event)
    output["dataset-2"][event] = {"captures": captures}

Write `output` to a JSON file:

In [128]:
!mkdir data

with open("data/simple-ff-test-1x2.json", "w") as f:
    json.dump(output, f)

mkdir: cannot create directory ‘data’: File exists
